## predicting Option price (Only European) by predicting stock price

In [5]:
#imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import exp, sqrt, log, e
from scipy.stats import norm
import datetime as dt


from pandas_datareader import data as wb


## Calculating the implied volatility 

In [6]:
def binomial_lattice(S0, K, r, v, T, n, call_put, exercise_policy):    
    time_step = T/n
    
    # Compute u and d
    """ Fill in appropriate formulas    DONE"""
    u = pow(e, v * sqrt(time_step))
    d = pow(e, -v * sqrt(time_step))
    
    # Compute p and q 
    """ Fill in appropriate formulas      DONE"""
    p = ((pow(e, r * time_step)) -d) / (u - d)
    q = 1 - p   
    
    # Create empty matrix for stock prices
    stock_price = np.zeros((n+1,n+1))
    
    # Set initial stock price
    stock_price[0,0] = S0
    
    # Fill matrix with stock prices per time step
    for i in range(1, n+1):
        stock_price[i,0] = stock_price[i-1,0]*u
        for j in range(1, i+1):
            stock_price[i,j] = stock_price[i-1,j-1]*d
    
    # Transform numpy matrix into Pandas Dataframe
    df_stock_price = pd.DataFrame(data=stock_price)
    df_stock_price = df_stock_price.T
    
    # Create empty matrix for option values
    option_value = np.zeros((n+1,n+1))
    
    # For final time step, compute option value based on stock price and strike price
    for i in range(n+1):
        if call_put == 'Call':
            """Fill in appropriate formula DONE"""
            option_value[n,i] = max(0, stock_price[n,i] - K)
        elif call_put == 'Put':
            """Fill in appropriate formula DONE"""
            option_value[n,i] =  max(0, K -  stock_price[n,i] )
            
    # Compute discount factor per time step
    """Fill in appropriate formula DONE"""
    discount = pow(e, -r * time_step)
    
    # Recursively compute option value at time 0
    if (call_put == 'Call' or call_put == 'Put') and exercise_policy == 'European':
        for i in range(n-1,-1,-1):
            for j in range(i+1):
                """ Fill in appropriate formulas for the different option types DONE"""
                option_value[i,j] = (option_value[i+1,j]*p + option_value[i+1,j+1] * q ) * discount
        
    elif call_put == 'Put' and exercise_policy == 'American':
        for i in range(n-1,-1,-1):
            for j in range(i+1):
                """ Fill in appropriate formulas for the different option types"""
                holdingPaper = (option_value[i+1,j]*p + option_value[i+1,j+1] * q ) * discount
                exerciseNow = K - stock_price[i,j]
                option_value[i,j] = max (holdingPaper, exerciseNow)

    
            
    df_option_value = pd.DataFrame(data=option_value)
    df_option_value = df_option_value.T
    
    return option_value[0,0], df_stock_price, df_option_value


def black_scholes_vega(S, K, T, r, v):
    d1 = (log(S / K) + (r + 0.5 * v ** 2) * T) / (v * sqrt(T))
    return S * norm.pdf(d1) * sqrt(T)

"""Implied volatility is found using the Newton-Raphson method"""
def compute_implied_volatility(true_price, S, K, r, T, n, call_put, exercise_policy):
    MAX_NO_ITERATIONS = 100
    MAX_VOL_UPDATE = 0.1
    ACCURACY = 1.0e-5

    implied_vol = .5 # Initial estimate for implied volatility 
    
    for i in range(MAX_NO_ITERATIONS):
        # Compute price with binomial lattice, using current estimate for implied volatility
        model_price, _, _ = binomial_lattice(S, K, r, implied_vol, T, n, call_put, exercise_policy)
        
        # Compute difference between model price and market price (the root) 
        diff = model_price - true_price

        # Terminate algorithm if desired precision has been hit 
        if (abs(diff) < ACCURACY):
            return implied_vol
        
        # Update implied volatility based on vega and observed error
        vega = black_scholes_vega(S, K, T, r, implied_vol)

        implied_vol -= np.clip(diff/vega, -MAX_VOL_UPDATE, MAX_VOL_UPDATE)
        
    # If maximum number of iterations is hit, simply return best estimate so far
    return implied_vol



## Predict the stock prices

In [55]:
#of One Stock
stock = 'TSLA'
end_date = dt.datetime.now()
start_date = end_date - dt.timedelta(days=3650)
time = 100 # how many days to predict
sims = 1000 #series of predictions

def monto_carlo_StockPrices(stock, start_date, end_date, sims, time, stock_price):

    data = pd.DataFrame()
    data[stock] = wb.DataReader(stock, data_source='yahoo', start=start_date, end=end_date)['Adj Close']
#     print(data)

    log_returns = np.log(1+data.pct_change())
#     print(log_returns)

    #all constants for daily returns with Drift
    u = log_returns.mean()
    var = log_returns.var()
    drift = u - (0.5*var)
    std = log_returns.std()
    #the daily_returns randomized
    daily_returns = np.exp(drift.values + std.values * norm.ppf(np.random.rand(time, sims)))

    # S0 = data.iloc[-1]
    #in stead of price list, we do change_list, because we want to know how the price changes over a certain amount of time 
    #excluding any price choose.
    # change_list = [1] * sims
    # for t in range(1, time):
    #     change_list = change_list*daily_returns[t]
    # change_list

    #just do price_list
    price_list = [stock_price] * sims
    for t in range(1, time):
        price_list = price_list*daily_returns[t]
    return price_list

monto_carlo_StockPrices(stock, start_date, end_date, sims, time)

array([ 81.66665075,  84.59856977, 146.06955132,  93.34671674,
       125.92790421, 185.2340167 ,  93.86563343,  82.92646282,
        68.8380298 ,  91.3668723 ,  90.97807362, 242.19504929,
        66.39425632, 142.2696791 , 166.02235207, 119.24184394,
       115.48610692, 152.32178019, 109.10348746, 143.36854265,
        80.84904904, 109.82157386, 199.04406908,  68.54225345,
        87.26094248, 111.56042117, 118.29668268, 115.31422949,
       166.00857409,  76.21092502,  98.07751601,  72.67314947,
       103.70270318, 102.03645847, 141.59702565,  90.33808551,
       115.25269256,  90.92911536, 156.60964747, 120.3765361 ,
        72.46396764, 144.95374573, 126.55029779, 184.23153321,
       133.40714985,  84.12457896,  91.98336962, 160.49196503,
       219.92168787, 152.58699218, 120.76940771, 189.74046606,
       135.55053967, 127.39184951, 130.41737042, 199.15638146,
        72.05198111, 139.29833021,  88.2689246 , 175.31716145,
       126.98173089,  99.92387598,  64.75131769, 127.17

## Calculate the average option price

In [58]:
#Calculate option price, because we have 'sims' price changes, every sim has a change of 1/sim, so:
#How to calculate probability?, this way is to calculate the average
strike_price = 100
stock_price = 100
option_policy = 'Call' #'Put'

def calculate_predicted_option (stock, start_date, end_date, sims, time, strike_price, stock_price,option_policy):
    price_list = monto_carlo_StockPrices(stock, start_date, end_date, sims, time, stock_price)

    if option_policy == 'Call':
        #european call:
        option_Price = 0
        for future_Price in price_list:
            if future_Price > strike_price:
                option_Price += (future_Price-strike_price)*1/sims
        return option_Price
    else: 
        #european Put:
        option_Price = 0
        for future_Price in price_list:
            if future_Price < strike_price:
                option_Price += (strike_price - future_Price)*1/sims
        return option_Price
    
stock = 'TSLA'
end_date = dt.datetime.now()
start_date = end_date - dt.timedelta(days=700)
time = 100 # how many days to predict
sims = 1000 #series of predictions
strike_price = 100
stock_price = 100
# option_policy = 'Call'
option_policy = 'Put' 

calculate_predicted_option (stock, start_date, end_date, sims, time, strike_price, stock_price,option_policy)

11.25414685467271